<a href="https://colab.research.google.com/github/UiinKim/Music_Generation/blob/main/data_extratct_for_youtube(MusicCaps).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

musicbench = load_dataset("amaai-lab/MusicBench")
musiccaps = load_dataset("google/MusicCaps")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
!pip install -U datasets[audio] #datasets 라이브러리 최신화[오디오]
!pip install yt-dlp #유튜브 비디오 다운로드 기능
!pip install gradio #interactive web application

  Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 11.0.3
    Uninstalling websockets-11.0.3:
      Successfully uninstalled websockets-11.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio-client 1.1.1 requires websockets<12.0,>=10.0, but you have websockets 12.0 which is incompatible.
  Using cached websockets-11.0.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached websockets-11.0.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014

In [4]:
from datasets import load_dataset

ds = load_dataset('google/MusicCaps', split='train') #train만 로드하여 저장

In [5]:
ds

Dataset({
    features: ['ytid', 'start_s', 'end_s', 'audioset_positive_labels', 'aspect_list', 'caption', 'author_id', 'is_balanced_subset', 'is_audioset_eval'],
    num_rows: 5521
})

In [6]:
import subprocess
import os
from pathlib import Path

In [7]:
ds[0]

{'ytid': '-0Gj8-vB1q4',
 'start_s': 30,
 'end_s': 40,
 'audioset_positive_labels': '/m/0140xf,/m/02cjck,/m/04rlf',
 'aspect_list': "['low quality', 'sustained strings melody', 'soft female vocal', 'mellow piano melody', 'sad', 'soulful', 'ballad']",
 'caption': 'The low quality recording features a ballad song that contains sustained strings, mellow piano melody and soft female vocal singing over it. It sounds sad and soulful, like something you would hear at Sunday services.',
 'author_id': 4,
 'is_balanced_subset': False,
 'is_audioset_eval': True}

In [8]:
# JUST HELPER METHODS IN THIS CELL

import subprocess
import os
from pathlib import Path

def download_clip( #유튜브에서 오디오 클립 다운로드 역할하는 함수
    video_identifier, #유튜브 비디오 id
    output_filename, #다운로드한 파일의 출력 경로
    start_time, #다운로드할 비디오의 시작 시간 위치
    end_time, #종료
    tmp_dir='/tmp/musiccaps', #임시 디렉토리
    num_attempts=5, #시도 횟수
    url_base='https://www.youtube.com/watch?v=' #유튜브 URL 베이스 경로
):
    status = False

    #유튜브 다운로드하는 yt-dlp 명령어 시행
    command = f"""
        yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" {url_base}{video_identifier}
    """.strip()

    attempts = 0 #시도 횟수 기록
    while True: #다운로드 시도
        try: #명령어 command 실행 및 출력 캡쳐
            output = subprocess.check_output(command, shell=True,
                                                stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)#파일이 존재하는지 확인
    return status, 'Downloaded' #존재여부와 메시지 반환

def process(example): #요구하는 예제(데이터셋에 존재하는)를 처리한 후 오디오 파일 다운로드
    outfile_path = str(data_dir / f"{example['ytid']}.wav") #출력 파일 경로
    status = True #다운로드 상태
    if not os.path.exists(outfile_path): #파일이 존재하지 않으면 다운로드
        status = False
        status, log = download_clip( #필요한 컬럼들 넣어서 함수 호출
            example['ytid'],
            outfile_path,
            example['start_s'],
            example['end_s'],
        )

    example['audio'] = outfile_path #데이터셋에 오디오 파일 경로 추가
    example['download_status'] = status #데이터셋에 다운로드 상태 추가
    return example

In [9]:
from datasets import Audio

samples_to_load = 32      # 데이터셋에서 로드할 오디오 파일의 수
cores = 4                 # 로딩에 사용할 프로세스의 수
sampling_rate = 44100     # 오디오 샘플링 속도
writer_batch_size = 1000  # 각 워커 당 메모리에 유지할 샘플 수
data_dir = "./music_data" # 데이터를 저장할 디렉토리

# Just select some samples
ds = ds.select(range(samples_to_load)) #해당 숫자만큼 샘플 선택

# Create directory where data will be saved
data_dir = Path(data_dir) #경로 객체 생성
data_dir.mkdir(exist_ok=True, parents=True) #디렉토리 생성, 이미 디렉토리가 있는 경우엔 오류 발생x

ds = ds.map(
        process, #map을 활용하여 process 함수를 여러 프로세스로 실행
        num_proc=cores, #사용할 프로세스의 수
        writer_batch_size=writer_batch_size, #워커 당 메모리에 유지할 수
        keep_in_memory=False #데이터를 메모리에 유지하지 않음
    ).cast_column('audio', Audio(sampling_rate=sampling_rate)) #audio column을 Audio 형식으로 변환

Map (num_proc=4):   0%|          | 0/32 [00:00<?, ? examples/s]

In [10]:
ds[0]

{'ytid': '-0Gj8-vB1q4',
 'start_s': 30,
 'end_s': 40,
 'audioset_positive_labels': '/m/0140xf,/m/02cjck,/m/04rlf',
 'aspect_list': "['low quality', 'sustained strings melody', 'soft female vocal', 'mellow piano melody', 'sad', 'soulful', 'ballad']",
 'caption': 'The low quality recording features a ballad song that contains sustained strings, mellow piano melody and soft female vocal singing over it. It sounds sad and soulful, like something you would hear at Sunday services.',
 'author_id': 4,
 'is_balanced_subset': False,
 'is_audioset_eval': True,
 'audio': {'path': 'music_data/-0Gj8-vB1q4.wav',
  'array': array([0.01234436, 0.01330566, 0.01473999, ..., 0.02119446, 0.01864624,
         0.01977539]),
  'sampling_rate': 44100},
 'download_status': True}

In [11]:
import gradio as gr

def get_example(idx):
    ex = ds[idx]
    return ex['audio']['path'], ex['caption']

gr.Interface(
    get_example,
    inputs=gr.Slider(0, len(ds) - 1, value=0, step=1),
    outputs=['audio', 'textarea'],
    allow_flagging="never",
    live=True
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6c82190cdbd4aff6c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
